In [18]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git@main

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-2jwxlzwf
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-2jwxlzwf
  Resolved https://github.com/huggingface/transformers to commit ba56ed0869eb4bbeb1c04af7f62a04350150e8d4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install git+https://github.com/openai/whisper.git -q -U

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.6 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#Input model
model_name_or_path = "openai/whisper-small"
language = "English"
language_abbr = "en"
task = "transcribe"
dataset_name = "sidd272/speechocean"

#output model path
org ="sushreesethi"
trained_adapter_name = "whisper-small-llm-lingo-adapters"
trained_model_name = "whisper-small-llm-lingo"
trained_adapter_repo = org + "/" + trained_adapter_name
trained_model_repo = org + "/" + trained_model_name

In [3]:
from transformers import pipeline
from transformers import (
    AutomaticSpeechRecognitionPipeline,
    WhisperTimeStampLogitsProcessor,
    WhisperForConditionalGeneration,
    WhisperTokenizer,
    WhisperProcessor
)
import torch

In [4]:
whisper_asr = pipeline(
    "automatic-speech-recognition",
    model=model_name_or_path,
    chunk_length_s =600,
    device="cuda" if torch.cuda.is_available() else "cpu"

)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [5]:
def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)  # Round seconds to integer
    milliseconds = int((seconds - int(seconds)) * 1000)  # Calculate milliseconds
    return f"{hours:02}:{minutes:02}:{seconds:02}.{milliseconds:03}"

def process_audio_and_create_vtt(audio_filename, audio_type, whisper_asr, output_filename=None):
    prediction = whisper_asr(f"{audio_filename}.{audio_type}", return_timestamps=True)

    if "chunks" not in prediction or not prediction["chunks"]:
        print("Whisper did not predict any chunks.")
        return

    vtt_file_name = output_filename if output_filename else f"{audio_filename}.vtt"

    with open(vtt_file_name, "w", encoding='utf-8') as vtt_file:
        vtt_file.write("WEBVTT\n\n")
        start_time = 0
        for chunk in prediction["chunks"]:
            end_time = format_time(chunk["timestamp"][1])
            text = chunk["text"]
            vtt_file.write(f"{format_time(start_time)} --> {end_time}\n{text}\n\n")
            start_time = chunk["timestamp"][1]  # Update start time for next chunk


In [6]:
uploaded_folder_path = '/content/train'
i=1
# Iterate over all files in the folder
for filename in os.listdir(uploaded_folder_path):
    # Check if the file is a regular file (not a directory)
     file_name = filename.split('.')[0]
     if(filename.split('.')[1]=='WAV'):
        print(filename)
        print(i)
        i=i+1
        process_audio_and_create_vtt(f'train/{file_name}', "WAV", whisper_asr)

000060136.WAV
1


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


000260112.WAV
2
000360283.WAV
3
000010122.WAV
4
000010115.WAV
5
000360133.WAV
6
000360378.WAV
7
000260050.WAV
8
000260069.WAV
9
000360360.WAV
10
000360223.WAV
11


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    a

000010133.WAV
12
000260075.WAV
13
000260011.WAV
14
000050118.WAV
15
000050038.WAV
16
000260052.WAV
17
000060077.WAV
18
000050122.WAV
19
000050100.WAV
20
000060106.WAV
21
000060029.WAV
22
000260166.WAV
23
000010063.WAV
24
000060102.WAV
25
000260133.WAV
26
000260126.WAV
27
000260091.WAV
28
000260096.WAV
29
000050024.WAV
30
000050114.WAV
31
000060056.WAV
32
000060094.WAV
33
000260121.WAV
34
000360190.WAV
35
000260032.WAV
36
000050049.WAV
37
000050010.WAV
38
000060081.WAV
39
000360314.WAV
40
000060031.WAV
41
000360132.WAV
42
000360334.WAV
43
000010145.WAV
44
000260095.WAV
45
000260048.WAV
46
000360210.WAV
47
000260001.WAV
48
000050055.WAV
49
000360034.WAV
50
000060113.WAV
51
000010106.WAV
52
000050174.WAV
53
000010173.WAV
54
000060083.WAV
55
000360332.WAV
56
000050003.WAV
57
000050028.WAV
58
000010075.WAV
59
000360241.WAV
60
000060116.WAV
61
000360343.WAV
62
000260015.WAV
63
000360161.WAV
64
000050095.WAV
65
000010035.WAV
66
000260115.WAV
67
000360313.WAV
68
000060015.WAV
69
000010053.WAV


In [7]:
uploaded_folder_path = '/content/test'
i=1
# Iterate over all files in the folder
for filename in os.listdir(uploaded_folder_path):
    # Check if the file is a regular file (not a directory)
     file_name = filename.split('.')[0]
     if(filename.split('.')[1]=='WAV'):
        print(filename)
        print(i)
        i=i+1
        process_audio_and_create_vtt(f'test/{file_name}', "WAV", whisper_asr)

000920149.WAV
1
000030051.WAV
2
000920136.WAV
3
000440126.WAV
4
000490157.WAV
5
000240320.WAV
6
000490175.WAV
7
000490052.WAV
8
000490047.WAV
9
000490117.WAV
10
000240152.WAV
11
000490002.WAV
12
000490088.WAV
13
000240116.WAV
14
000440045.WAV
15
000240099.WAV
16
000240060.WAV
17
000920126.WAV
18
000240031.WAV
19
000030119.WAV
20
000440175.WAV
21
000030059.WAV
22
000030154.WAV
23
000920002.WAV
24
000440089.WAV
25
000240248.WAV
26
000920092.WAV
27
000240287.WAV
28
000240350.WAV
29
000030040.WAV
30
000240151.WAV
31
000440062.WAV
32
000490017.WAV
33
000030175.WAV
34
000920104.WAV
35
000920122.WAV
36
000440145.WAV
37
000440147.WAV
38
000030153.WAV
39
000030080.WAV
40
000490032.WAV
41
000490097.WAV
42
000030049.WAV
43
000240323.WAV
44
000490086.WAV
45
000240284.WAV
46
000920167.WAV
47
000920129.WAV
48
000240329.WAV
49
000920010.WAV
50
000440049.WAV
51
000490134.WAV
52
000240073.WAV
53
000030012.WAV
54
000920009.WAV
55
000240010.WAV
56
000030116.WAV
57
000490162.WAV
58
000030069.WAV
59
000440

In [8]:
import os

def get_files_in_directory(directory):
    files = os.listdir(directory)
    return files

def filter_files_by_extension(files, extension, directory):
    return sorted([os.path.join(directory, file) for file in files if file.endswith(extension)])

def get_wav_and_vtt_files(directory):
    files = get_files_in_directory(directory)
    wav_files = filter_files_by_extension(files, '.WAV', directory)
    vtt_files = filter_files_by_extension(files, '.vtt', directory)
    return wav_files, vtt_files

# Paths to your train and test directories containing WAV and VTT files
train_directory = '/content/train'
test_directory = '/content/test'

# Get WAV and VTT files for train and test datasets
train_wav_files, train_vtt_files = get_wav_and_vtt_files(train_directory)
test_wav_files, test_vtt_files = get_wav_and_vtt_files(test_directory)

# Print the lists of WAV and VTT files with complete paths
print("Train WAV files:", train_wav_files)
print("Train VTT files:", train_vtt_files)
print("Test WAV files:", test_wav_files)
print("Test VTT files:", test_vtt_files)


Train WAV files: ['/content/train/000010011.WAV', '/content/train/000010035.WAV', '/content/train/000010053.WAV', '/content/train/000010063.WAV', '/content/train/000010069.WAV', '/content/train/000010075.WAV', '/content/train/000010089.WAV', '/content/train/000010095.WAV', '/content/train/000010106.WAV', '/content/train/000010113.WAV', '/content/train/000010115.WAV', '/content/train/000010121.WAV', '/content/train/000010122.WAV', '/content/train/000010133.WAV', '/content/train/000010135.WAV', '/content/train/000010140.WAV', '/content/train/000010145.WAV', '/content/train/000010149.WAV', '/content/train/000010168.WAV', '/content/train/000010173.WAV', '/content/train/000050003.WAV', '/content/train/000050010.WAV', '/content/train/000050024.WAV', '/content/train/000050028.WAV', '/content/train/000050038.WAV', '/content/train/000050040.WAV', '/content/train/000050047.WAV', '/content/train/000050049.WAV', '/content/train/000050055.WAV', '/content/train/000050078.WAV', '/content/train/000050

In [9]:
pip install webvtt-py

  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=07e9c14f90f0760debcbbf79efa1d369bb7cdd97f4a66c8374536d0cb6f678f6
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [30]:
import os
import librosa
import webvtt

# Function to load WAV and VTT files using librosa and webvtt
def load_data(wav_files, vtt_files):
    dataset = []
    for wav_file, vtt_file in zip(wav_files, vtt_files):
        # Load audio file using librosa
        audio, sr = librosa.load(wav_file, sr=None)

        # Load corresponding VTT file using webvtt
        captions=""
        for caption in webvtt.read(vtt_file):
            captions=caption.text
        dataset.append({'speech': audio, 'captions': captions})

    return dataset

# Load data
train_dataset = load_data(train_wav_files, train_vtt_files)
test_dataset = load_data(test_wav_files, test_vtt_files)

# Print dataset info
print("Train dataset length:", len(train_dataset))
print("Test dataset length:", len(test_dataset))

# Example data from train dataset
print("Example data from train dataset:")
print(train_dataset[0])

# Example data from test dataset
print("Example data from test dataset:")
print(test_dataset[0])


Train dataset length: 100
Test dataset length: 100
Example data from train dataset:
{'speech': array([-9.4604492e-04, -2.3803711e-03, -1.3122559e-03, ...,
       -9.1552734e-05,  3.0517578e-04, -2.4414062e-04], dtype=float32), 'captions': ' We call it bear.'}
Example data from test dataset:
{'speech': array([-0.00119019, -0.00500488, -0.00283813, ...,  0.00274658,
        0.        ,  0.00125122], dtype=float32), 'captions': ' Mark is going to see elephant.'}


In [31]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [32]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [33]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [34]:
import pandas as pd
from datasets import Dataset
train_dataset=pd.DataFrame(train_dataset)
train_dataset = Dataset.from_pandas(train_dataset)
test_dataset=pd.DataFrame(test_dataset)
test_dataset = Dataset.from_pandas(test_dataset)


In [35]:
train_dataset[0]

{'speech': [-0.000946044921875,
  -0.00238037109375,
  -0.001312255859375,
  -0.001007080078125,
  -0.001129150390625,
  0.000213623046875,
  0.000457763671875,
  0.0010986328125,
  0.000213623046875,
  0.000732421875,
  0.0001220703125,
  -0.000640869140625,
  0.0003662109375,
  -0.00018310546875,
  0.000152587890625,
  0.000762939453125,
  0.000518798828125,
  0.001007080078125,
  0.00079345703125,
  0.000274658203125,
  0.00048828125,
  0.0003662109375,
  0.001007080078125,
  0.001190185546875,
  -0.00067138671875,
  -0.0008544921875,
  -0.0008544921875,
  -0.000732421875,
  -0.00079345703125,
  -0.00018310546875,
  -0.000823974609375,
  -0.001251220703125,
  -0.00018310546875,
  -0.000244140625,
  -0.00042724609375,
  9.1552734375e-05,
  3.0517578125e-05,
  0.0,
  0.0001220703125,
  -0.00030517578125,
  0.000396728515625,
  0.000640869140625,
  0.000335693359375,
  0.00018310546875,
  -0.00030517578125,
  0.000244140625,
  0.000457763671875,
  -0.000274658203125,
  0.00064086914062

In [36]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["speech"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio, sampling_rate=16000).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["captions"]).input_ids
    return batch

In [37]:
train_dataset = train_dataset.map(prepare_dataset,remove_columns=train_dataset.column_names, num_proc=1)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [38]:
test_dataset = test_dataset.map(prepare_dataset,remove_columns=test_dataset.column_names, num_proc=1)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [39]:
test_dataset[0]

{'input_features': [[-0.5116540193557739,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049,
   -0.6108067035675049

In [40]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [41]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [42]:
import evaluate

metric = evaluate.load("wer")

In [43]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [44]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, load_in_8bit=True, device_map="auto")

# model.hf_device_map - this should be {" ": 0}

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [45]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [46]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 245,273,856 || trainable%: 1.442854145857274


In [54]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="temp",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-3,
    warmup_steps=50,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    fp16=True,
    per_device_eval_batch_size=8,
    generation_max_length=128,
    logging_steps=25,
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],  # same reason as above
)

In [57]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [58]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,0.388324
2,0.021900,0.386096
3,0.021900,0.373315


TrainOutput(global_step=39, training_loss=0.02772385646135379, metrics={'train_runtime': 177.5375, 'train_samples_per_second': 1.69, 'train_steps_per_second': 0.22, 'total_flos': 8.8104443904e+16, 'train_loss': 0.02772385646135379, 'epoch': 3.0})